# Enhancing Regression: Categorical Preprocessing, Interactions and Polynomials

![kneeding](https://media.giphy.com/media/RpckSiHL6ZaXS/giphy.gif)

Objectives, be able to:

1. Preprocess non-numeric data:
> - categorical: get_dummies/one-hot-encoder
> - binary encoder
> - ordinal: label encoder


2. Feature Engineer:
> - Interaction terms
> - Polynomials

## Scenario: car seat sales

We will continue to work with our car seat data.  <br>
Here, again are all of the features.

- Sales: unit sales at each location
- CompPrice: price charged by nearest competitor at each location
- Income: community income level
- Advertising: local advertising budget for company at each location
- Population: population size in region (in thousands)
- Price: price charged for car seat at each site
- ShelveLoc: quality of shelving location at site (Good | Bad | Medium)
- Age: average age of the local population
- Education: education level at each location
- Urban: whether the store is in an urban or rural location
- USA: whether the store is in the US or not

In [1]:
# import necessary libraries
%matplotlib inline
import warnings
warnings.filterwarnings(action='ignore')

%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd
df = pd.read_csv('data/Carseats.csv')
df.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


In [4]:
from sklearn.model_selection import train_test_split
from continuous_prep import scale_numeric
X = df.drop('Sales', axis=1)
y = df['Sales']

X_numeric = X.select_dtypes(exclude='object')

# Preprocess numeric with custom function
X_numeric, y = scale_numeric(X_numeric, y)

> Take a moment to look through the continuous_prep.py file.  Add comments to the file to communicate its function  

In [5]:
# Isolate object type and match index to preprocessed y
X_cat = X.select_dtypes(include='object')
X_cat = X_cat.iloc[X_numeric.index]

## Different types of categorical features
> We have three categorical features, ShelveLoc, Urban, and US.
>  - How would you expect to translate these categories into input that a machines are familiar with?

In [6]:
# your answer here

## Label Binarizer


Urban and US are binary, Yes/No, so let's handle them first.
From sklearn.preprocessing, we can import a module called LabelBinarizer


In [7]:
from sklearn.preprocessing import LabelBinarizer
# Label Binarizer uses familiar syntax.
us_lb = LabelBinarizer()
X_cat["US"] = us_lb.fit_transform(X_cat["US"])

In [9]:
# your turn. Do the same with Urban


## Dummy Variables and One-Hot-Encoder

Now let's deal with ShelveLoc. We have a couple of ways we could handle it. The first way we will discuss is the poorly named dummy variables.  


What is a dummy/one-hot-encoded variable?

> We create dummy variables from categorical features with multiple categories.  Each unique category is transformed into its own column filled with 0's and 1's.  The rows with 1's indicate that the row is associated with that category.  When we fit our model, we create coefficients for each column.

In [10]:
# Let's take a look at the ShelveLoc feature

In [11]:
X_cat.ShelveLoc

0         Bad
1        Good
2      Medium
3      Medium
4         Bad
        ...  
395      Good
396    Medium
397    Medium
398       Bad
399      Good
Name: ShelveLoc, Length: 387, dtype: object

In [12]:
# We can use pandas to "get_dummies"

pd.get_dummies(X_cat.ShelveLoc)

,Bad,Good,Medium
0,1,0,0
1,0,1,0
2,0,0,1
3,0,0,1
4,1,0,0
...,...,...,...
395,0,1,0
396,0,0,1
397,0,0,1
398,1,0,0


_Footnote_<br>
We can also use **One Hot Encoder**. OneHotEncoder has the advantage of the .fit() method.  The encoder can then be saved and used on future data.

In [13]:
# save for label encoder
X_cat_for_le = X_cat.copy()

## Dummy Variable Trap

![](https://media.giphy.com/media/8McNH1aXZnVyE/giphy.gif)

> Take a minute to think about the ShelveLoc dummy-variable example.  If you were to remove one column, would you lose any information?  

> If we took out the "Bad" column, is there a way we could still figure out which records had "Bad" shelf location?

> Discuss with your neighbors.

- The dummy variable trap is a problem with multicollinearity.  We can create one column from a combination of the other columns.  By creating dummies in this way, we  violate an assumption of linear regression.  

- When features have high collinearity, the coefficients become difficult to interpret.  Because they are collinear, the beta coefficients will be highly influenced by the presence of the other feature.

- To address this problem, we drop one column.  Get dummies has parameter "drop_first" to address this.

In [14]:
# use get dummies to add two columns, "Good" and "Medium"
# to our dataframe.  Drop the original ShelveLoc column.


In [15]:
X_cat = X_cat.join(pd.get_dummies(X_cat['ShelveLoc'], drop_first = True))
X_cat.drop('ShelveLoc', axis=1, inplace=True)

In [16]:
X_cat.head()

,Urban,US,Good,Medium
0,1,1,0,0
1,1,1,1,0
2,1,1,0,1
3,1,1,0,1
4,1,0,0,0


In [17]:
from sklearn.linear_model import LinearRegression
# Let's rejoin our categorical and numerical columns
# then fit our model again.

X = X_numeric.join(X_cat)
lr = LinearRegression()
lr.fit(X, y)
lr.score(X, y)


0.8674167170661751

Another option would be to apply Label Encoder.  Perhaps we have prior knowledge about the relative value of Bad, Mediumm, and Good shelf locations.  Perhaps we think that Medium is twice as important as Bad, and Good is 3 times as important as Bad.  We can then do the following:

Fitting a linear regression to the data encoded in this way treats the feature as a continuous independent variable.  If our domain knowledge is good, then such a fit could perform better than the dummy variable.  You can try out both ways and see the effect on the metrics.

Just for fun, let's fit our model with the bgm encoded values and see what kind of R-squared it returns.

In [18]:
def bgm_encoder(element):
    if element == 'Bad':
        return 0
    elif element == 'Medium':
        return 1
    else:
        return 2

X_example = X_numeric.join(X_cat_for_le)
X_example['ShelveLoc'] = X_example['ShelveLoc'].apply(bgm_encoder)
lr.fit(X_example, y)
lr.score(X_example, y)

0.8606526343461581

> Discussion: Can you imagine a scenario when label encoding could be highly effective? Consider the domain knowledge of the model designer.

## Interaction Terms
An interaction term is a feature which accounts for the join affect of two other features that is non-additive.  Non-additive is the important distinction here, since our linear model is based on additive relationships of our features: the features in the data set add together to predict the target.

The plot below shows an example of a non-additive relationship: the effect of a drug vs a placebo on people who experienced different severities of stroke.  Based on the three groups of strokes, the plot shows the slopes (i.e. the change in effectiven of the drug across drug and placebo treatments)  to be equivalent between mild and moderate stroke victims, but markedly different for severe stroke victims. 

![](img/stroke_plot.png)

The difference in slope in one group indicates that   survival rate and type of treatment does not change equally across groups.  In other words, the effect is not additive with regards to stroke severity.  In order to capture that relationship, we create an interaction term between stroke severity and treatment.  An interaction term is just two features multiplied together.

To explore interaction terms, we will import the mtcars dataset and fit a vanilla linear regression model: 

In [365]:
mt_cars = pd.read_csv('data/mtcars.csv')
mt_cars.head()

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [366]:
mt_cars = mt_cars.drop('model', axis=1)
y = mt_cars['mpg']
X = mt_cars.drop('mpg', axis=1)

In [367]:
lr = lr.fit(X,y)
base_score = lr.score(X,y)
print(base_score)

0.8690157644777647


In [368]:
data = X.join(y)
formula = 'mpg ~ ' + '+'.join(X.columns)
mod = smf.ols(formula=formula, data = data)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.869
Model:                            OLS   Adj. R-squared:                  0.807
Method:                 Least Squares   F-statistic:                     13.93
Date:                Tue, 25 Feb 2020   Prob (F-statistic):           3.79e-07
Time:                        22:03:04   Log-Likelihood:                -69.855
No. Observations:                  32   AIC:                             161.7
Df Residuals:                      21   BIC:                             177.8
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     12.3034     18.718      0.657      0.518     -26.623      51.229
cyl           -0.1114      1.045     -0.107      0.916      -2.285       2.062
disp           0.0133      0.018      0.747      0.463      -0.024       0.050
hp            -0.0215      0.022     -0.987      0.335      -0.067       0.024
drat           0.7871      1.635      0.481      0.635      -2.614       4.188
wt            -3.7153      1.894     -1.961      0.063      -7.655       0.224
qsec           0.8210      0.731      1.123      0.274      -0.699       2.341
vs             0.3178      2.105      0.151      0.881      -4.059       4.694
am             2.5202      2.057      1.225      0.234      -1.757       6.797
gear           0.6554      1.493      0.439      0.665      -2.450       3.761
carb          -0.1994      0.829     -0.241      0.812      -1.923       1.524
==============================================================================
Omnibus:                        1.907   Durbin-Watson:                   1.861
Prob(Omnibus):                  0.385   Jarque-Bera (JB):                1.747
Skew:                           0.521   Prob(JB):                        0.418
Kurtosis:                       2.526   Cond. No.                     1.22e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.22e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [370]:
# We can create an interaction term by simply multiplying two features together:
X['cyl_disp'] = X['cyl'] * X['disp']
data = X.join(y)
formula = 'mpg ~ ' + '+'.join(X.columns)
mod = smf.ols(formula=formula, data = data)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.898
Model:                            OLS   Adj. R-squared:                  0.841
Method:                 Least Squares   F-statistic:                     15.94
Date:                Tue, 25 Feb 2020   Prob (F-statistic):           1.44e-07
Time:                        22:05:40   Log-Likelihood:                -65.916
No. Observations:                  32   AIC:                             155.8
Df Residuals:                      20   BIC:                             173.4
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     29.9764     18.535      1.617      0.121      -8.687      68.640
cyl           -1.7896      1.184     -1.512      0.146      -4.259       0.679
disp          -0.0959      0.049     -1.958      0.064      -0.198       0.006
hp            -0.0334      0.020     -1.641      0.116      -0.076       0.009
drat          -0.5412      1.585     -0.342      0.736      -3.847       2.765
wt            -3.5527      1.718     -2.068      0.052      -7.136       0.030
qsec           0.6981      0.664      1.051      0.306      -0.687       2.084
vs             0.8287      1.919      0.432      0.670      -3.174       4.832
am             0.8191      1.998      0.410      0.686      -3.348       4.986
gear           1.5545      1.405      1.106      0.282      -1.377       4.486
carb           0.1442      0.765      0.189      0.852      -1.451       1.740
cyl_disp       0.0138      0.006      2.363      0.028       0.002       0.026
==============================================================================
Omnibus:                        1.577   Durbin-Watson:                   2.059
Prob(Omnibus):                  0.455   Jarque-Bera (JB):                1.219
Skew:                           0.259   Prob(JB):                        0.544
Kurtosis:                       2.196   Cond. No.                     8.81e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.81e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

That had a good effect on our r_squared. 

>Now, it is your turn to explore the data and find the most influential interaction turn.  Hint: Try using itertools combinations.

In [19]:
from itertools import combinations
# Your code here

## Polynomial Terms

Another way to potentially improve our score is to account for non-linear relationships between target and feature.  Polynomial transformations, for example a quadratic relationship, can account for curved relationships.


Take for example the relationship between age and income shown it the plot below:

![](img/age_v_income.png)

The curved shape of these relationships will not be captured by a simple linear relationship between age and income. Income increases with age until around 50, and then starts to decrease. When seeing a curve like this, you should consider that the addition of a polynomial term will capture an otherwise overlooked relationship between feature and target.

Let's use the famous Boston Housing dataset to practice identifying polynomial relationships.

The dataset has been imported and a basic model fit with an R^2 of ~.74.


In [310]:
import sklearn.datasets
boston = sklearn.datasets.load_boston()
X = boston.data
X = pd.DataFrame(X)
X.columns = boston.feature_names
y = pd.DataFrame(boston.target)

lr = LinearRegression()
lr.fit(X,y)
lr.score(X,y)

0.7406426641094095

Your task is to explore the relationships between targets and variables, and see whether the addition of any polynomial terms will benefit the model substantially.

In [21]:
import seaborn as sns
# your code here
# hint, use pairplot

We can also apply the PolynomialFeature method from sklearn, which creates polynomial terms of a specified degree for each feature. 

In [350]:
from sklearn.preprocessing import PolynomialFeatures

X = boston.data
X = pd.DataFrame(X)
X.columns = boston.feature_names
y = pd.DataFrame(boston.target)

pf = PolynomialFeatures(2, include_bias=False)

poly_df = pd.DataFrame(pf.fit_transform(X))
poly_df.columns = pf.get_feature_names(X.columns)
poly_df.head()

lr.fit(poly_df, y)
lr.score(poly_df, y)

0.9289961714593022

Polynomials have diminishing returns!

![polynomials](https://sc.cnbcfm.com/applications/cnbc.com/resources/files/2015/12/11/emotionandincome-01_0.png)

The sklearn polynomial feature creates a ton of features. R-2 always increases with more features.  This will lead to overfitting, which will lead to poor performance on data which has not been seen. 